---

## **<font color="Dodgerblue"> Modelación Climática Tropical</font>**
### **<font color="crimson"> Ciclo anual MSWEP</font>**


---

In [1]:
pip install xarray

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cftime

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install h5netcdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install netCDF4 

In [ ]:
pip install h5py netCDF4 h5netcdf

In [ ]:
pip install cartopy

In [ ]:
import xarray as xr
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import matplotlib.ticker as ticker

In [ ]:
def lineplot_setup(xmin,xmax,xlab,ylab,ymin=None,ymax=None,title=None,legend=False):
    plt.xlim([xmin,xmax])
    plt.ylim([ymin,ymax])
    plt.grid(linestyle='--',alpha=0.66)
    plt.minorticks_on()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(xlab,fontsize=14)
    plt.ylabel(ylab,fontsize=14)
    plt.title(title,fontsize=15)
    if legend:
        plt.legend(fancybox=True,shadow=True,fontsize=12.5)

In [ ]:
# Archivo de precipitación MSWEP
filename='/home/jovyan/ModClim/datos_actualizados/MSWEP_monthly_1980-2023.nc'
ds=xr.open_dataset(filename)
pr=ds['precipitation']
#pr.to_netcdf('MSWEP_Global.nc') # Guardar datos nc en caso de que no abra
pr

In [ ]:
pr.time

In [ ]:
# Convertimos las unidades
pr=pr*8
pr['units']='mm day-1'

In [ ]:
pr

In [ ]:
pr.units

In [ ]:
pr_years = pr.sel(time=slice('1980', '2023'))

In [ ]:
jjas = pr_years.where((pr.time.dt.month >= 6) & (pr.time.dt.month <= 9), drop=True)

In [ ]:
mean_jjas = jjas.mean('time')

In [ ]:
mean_jjas

In [ ]:
mean_jjas.min()

In [ ]:
mean_jjas.max()

### **<font color="Navy"> Ciclos anuales sobre el centro-sur de México </font>**


In [ ]:
# Asignamos pesos con la función coseno
lats = pr_years.lat
weights = np.cos(np.deg2rad(lats))
weights.name = "weights"

In [ ]:
# Aplicamos el peso y luego el promedio espacial
pr_weighted = pr_years.weighted(weights)
pr_meanglobal = pr_weighted.mean(dim=['lat', 'lon'])

In [ ]:
# Recortamos el centro-sur
centrolat=pr_years.sel(lat=slice(13.75,23.75))
centrolon=centrolat.sel(lon=slice(253.75,273.5))

centro_peso=centrolon.weighted(weights)
prcentro=centrolon.mean(["lon","lat"])

mensual_prcentro = prcentro.groupby(prcentro.time.dt.month).mean()
#mensual_prcentro.to_netcdf('MSWEP_ciclo_actualizado.nc')

In [ ]:
meses=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']

In [ ]:
plt.figure(figsize=(10,5), dpi=300)
plt.plot(meses,mensual_prcentro,c='steelblue',label='MSWEP (1980-2023)') 
lineplot_setup(None,None,'Mes','Precipitación [mm día$^{-1}$]',title='Ciclo anual de la precipitación en el centro-sur de México')
plt.legend()
plt.tight_layout()

In [ ]:
# Figura con todos los ciclos anuales
sns.set_style("whitegrid")

# Cargamos datos de cada archivo
ds_MSWEP = xr.open_dataset('MSWEP_ciclo_actualizado.nc')
ds_GPCP = xr.open_dataset('GPCP_ciclo_actualizado.nc')
ds_GPCC = xr.open_dataset('GPCC_ciclo_actualizado.nc')
ds_SPEAR1 = xr.open_dataset('SPEAR21_ciclo.nc')
ds_SPEAR2 = xr.open_dataset('SPEAR80_ciclo.nc')


# Extraemos la variable, cambia en cada caso
mswep = ds_MSWEP['precipitation']
gpcp = ds_GPCP['precip']
gpcc = ds_GPCC['precip']
spear = ds_SPEAR1['precip']
spear_2 = ds_SPEAR2['precip']


plt.figure(figsize=(10, 5), dpi=500)
# Graficamos cada uno
plt.plot(meses, gpcp, c='#ff7f00', label='GPCP (1979-2024)')
plt.plot(meses, mswep, c='mediumslateblue', label='MSWEP (1980-2023)')
#plt.plot(meses, spear, c='dodgerblue', label='SPEAR (1921-2010)') En caso de querer observar el ciclo completo
plt.plot(meses, spear_2, c='crimson', label='SPEAR (1980-2010)')
plt.plot(meses, gpcc, c='#4daf4a', label='GPCC (1982-2024)')


plt.xlabel('Mes',fontsize=13)
plt.ylabel('Precipitación [mm día$^{-1}$]',fontsize=13)
plt.title('Ciclo anual de la precipitación en el centro-sur de México',fontsize=15, pad=10, x=0.5)
plt.xticks(meses, ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],fontsize=11)
plt.legend()
plt.grid(True, which='major', linestyle='-', linewidth=0.8, alpha=0.7) 
plt.grid(True, which='minor', linestyle='-.', linewidth=0.5, alpha=0.5)
plt.minorticks_on() 
ax = plt.gca()  
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))  # Intervalos de 1 en 1
plt.tight_layout()
plt.show()